In [2]:
import torch
import torch.nn as nn
import pandas as pd 
import numpy as np

In [3]:
df=pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [4]:
print(f"No. of null values {df.isnull().sum()}")
print(f"No. of duplicate values {df.duplicated().sum()}")

No. of null values Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64
No. of duplicate values 0


In [81]:
df['Outcome'].value_counts(normalize="columns")*100

Outcome
0    65.104167
1    34.895833
Name: proportion, dtype: float64

In [7]:
df.shape

(768, 9)

In [5]:
X=df.drop(columns=['Outcome']).values
y=df['Outcome'].values

In [6]:
print(X.shape)
print(y.shape)

(768, 8)
(768,)


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=4)

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(614, 8)
(614,)
(154, 8)
(154,)


In [63]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [64]:
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test,dtype=torch.float32)

In [65]:
from torch.utils.data import Dataset,DataLoader

In [66]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features=features
        self.labels=labels
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self,index):
        # Here we can apply the transformation
        return self.features[index],self.labels[index]


In [67]:
train_dataset=CustomDataset(X_train_tensor,y_train_tensor)
test_dataset=CustomDataset(X_test_tensor,y_test_tensor)

In [82]:
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=False)
test_dataloader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [83]:
class Net(nn.Module):
    def __init__(self,input_shape ):
        super().__init__()
        self.Network=nn.Sequential(
            nn.Linear(in_features=input_shape,out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32,out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16,out_features=9),
            nn.ReLU(),
            nn.Linear(in_features=9,out_features=1),
            nn.Sigmoid()
        )
    def forward(self,features):
        out=self.Network(features)
        return out

In [84]:
model=Net(X_train_tensor.shape[1])
# model(X_train_tensor)

In [85]:
epochs=100
learning_rate=0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.BCELoss()

In [86]:
for epoch in range(epochs):
    for batch_features,batch_lables in train_dataloader:
        y_pred=model(batch_features)
        loss=loss_function(y_pred,batch_lables.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epochs: {epoch}, loss: {loss:.4f}")
        

Epochs: 0, loss: 0.5315
Epochs: 1, loss: 0.4595
Epochs: 2, loss: 0.4464
Epochs: 3, loss: 0.4256
Epochs: 4, loss: 0.4083
Epochs: 5, loss: 0.3780
Epochs: 6, loss: 0.3525
Epochs: 7, loss: 0.3311
Epochs: 8, loss: 0.3056
Epochs: 9, loss: 0.2742
Epochs: 10, loss: 0.2597
Epochs: 11, loss: 0.2172
Epochs: 12, loss: 0.2172
Epochs: 13, loss: 0.1845
Epochs: 14, loss: 0.1659
Epochs: 15, loss: 0.1835
Epochs: 16, loss: 0.1199
Epochs: 17, loss: 0.0899
Epochs: 18, loss: 0.0964
Epochs: 19, loss: 0.0634
Epochs: 20, loss: 0.0763
Epochs: 21, loss: 0.0557
Epochs: 22, loss: 0.0581
Epochs: 23, loss: 0.0826
Epochs: 24, loss: 0.0647
Epochs: 25, loss: 0.0662
Epochs: 26, loss: 0.0735
Epochs: 27, loss: 0.0395
Epochs: 28, loss: 0.0468
Epochs: 29, loss: 0.0306
Epochs: 30, loss: 0.0445
Epochs: 31, loss: 0.2705
Epochs: 32, loss: 0.2037
Epochs: 33, loss: 0.1814
Epochs: 34, loss: 0.0369
Epochs: 35, loss: 0.0272
Epochs: 36, loss: 0.0225
Epochs: 37, loss: 0.0447
Epochs: 38, loss: 0.0773
Epochs: 39, loss: 0.0137
Epochs: 40

In [87]:
# Initialize accuracy variables
total_correct = 0
total_samples = 0

# Iterate over the test dataloader
with torch.no_grad():
    for batch_features, batch_labels in test_dataloader:
        # Get predictions
        y_pred = model(batch_features)
        
        # Convert predictions to class labels (0 or 1)
        # predicted_labels = torch.round(y_pred)
        predicted_labels = (y_pred > 0.1).float()
        
        # Calculate correct predictions
        correct_predictions = (predicted_labels == batch_labels).sum()
        
        # Update accuracy variables
        total_correct += correct_predictions.item()
        total_samples += batch_labels.shape[0]

# Calculate accuracy
accuracy = total_correct / total_samples
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 15.4805


#### Less accuracy maybe because of imbalance Dataset